In [1]:
# Sasha Morrison, 2.5 Submission

import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.datasets import load_boston
import pandas as pd
import io
import numpy as np

In [2]:
#initialize
data_url = r'data/DBData.csv'
dataframe = pd.read_csv(data_url)
df = dataframe
# remove excess 
df = df.drop(['Indicator Code', 'Unnamed: 20', 'Country Code'], axis=1)
df2 = pd.DataFrame(
        df.pivot(index='Indicator Name', 
                  columns='Country Name')
         .unstack()
         .reset_index(drop=False)
)
df2.columns = ['Year', 'Country', 'Measure','Score']
df2

,Year,Country,Measure,Score
0,2004,Afghanistan,Dealing with construction permits (DB06-15 met...,NaN
1,2004,Afghanistan,Dealing with construction permits (DB16-19 met...,NaN
2,2004,Afghanistan,Dealing with construction permits: Building qu...,NaN
3,2004,Afghanistan,Dealing with construction permits: Building qu...,NaN
4,2004,Afghanistan,Dealing with construction permits: Cost (% of ...,NaN
...,...,...,...,...
695355,2019,Zimbabwe,Trading across borders: Time to export: Border...,45.07
695356,2019,Zimbabwe,Trading across borders: Time to export: Docume...,42.01
695357,2019,Zimbabwe,Trading across borders: Time to import (days) ...,NaN
695358,2019,Zimbabwe,Trading across borders: Time to import: Border...,18.76


In [3]:
df2 = df2.pivot_table(index=['Year', 'Country'], columns='Measure', values='Score').reset_index()
df2

Measure,Year,Country,Dealing with construction permits (DB06-15 methodology) - Score,Dealing with construction permits (DB16-19 methodology) - Score,Dealing with construction permits: Building quality control index (0-15) (DB16-19 methodology),Dealing with construction permits: Building quality control index (0-15) (DB16-19 methodology) - Score,Dealing with construction permits: Cost (% of Warehouse value),Dealing with construction permits: Cost (% of Warehouse value) - Score,Dealing with construction permits: Liability and insurance regimes index (0-2) (DB16-19 methodology),Dealing with construction permits: Procedures (number),...,Trading across borders: Documents to export (number) (DB06-15 methodology),Trading across borders: Documents to export (number) (DB06-15 methodology) - Score,Trading across borders: Documents to import (number) (DB06-15 methodology),Trading across borders: Documents to import (number) (DB06-15 methodology) - Score,Trading across borders: Time to export (days) (DB06-15 methodology) - Score,Trading across borders: Time to export: Border compliance (hours) (DB16-19 methodology) - Score,Trading across borders: Time to export: Documentary compliance (hours) (DB16-19 methodology) - Score,Trading across borders: Time to import (days) (DB06-15 methodology) - Score,Trading across borders: Time to import: Border compliance (hours) (DB16-19 methodology) - Score,Trading across borders: Time to import: Documentary compliance (hours) (DB16-19 methodology) - Score
0,2004,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3020,2019,Vietnam,NaN,79.05,12.0,80.00,0.7,96.54,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,66.04,71.01,NaN,80.29,68.62
3021,2019,West Bank and Gaza,NaN,56.15,12.0,80.00,14.4,28.24,0.0,20.0,...,NaN,NaN,NaN,NaN,NaN,96.86,57.99,NaN,98.21,81.45
3022,2019,"Yemen, Rep.",NaN,0.00,NaN,0.00,NaN,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,0.00,0.00
3023,2019,Zambia,NaN,71.65,10.0,66.67,2.6,86.92,0.0,10.0,...,NaN,NaN,NaN,NaN,NaN,25.16,43.79,NaN,57.35,70.29


# 1. EDB Score 

In `data/DBData.csv`, you have the full "ease of doing business" dataset from the World Bank. Reformat it into the **Tidy Data** format, so one row is per-year-per-country

Result should look like:

![](EDB_unstack.png)

# 2 GDP and ease of doing business

Using the additional data in `data/GDPpc.csv`, join the clean dataset in **1** to the GDP data.

**What are the 3 Ease of Doing Business variables most closely linked to GDP?**

Answer by giving their correlation ratio, and give a possible explanation and a data visualization

**hint:** trying to do `df.corr()` or `sns.pairplot()` on the whole dataset will crash most computers. Be smart about the number of columns you're testing at once.

In [4]:
data_url = r'data/GDPpc.csv'
dataframe = pd.read_csv(data_url)
df = dataframe
df = df.drop(['Country Code', 'Indicator Code'], axis=1)
df = pd.DataFrame(
        df.pivot(index='Indicator Name', 
                  columns='Country Name')
         .unstack()
         .reset_index(drop=False)
)
df = df.drop('Indicator Name', axis=1)
# df.pivot_table(index=['Year', 'Country'], columns='Measure', values='Score').reset_index()
df.columns = ['Year', 'Country', 'GDP/Cap(USD)']
df3 = df2.merge(df, on=['Year', 'Country'], how='inner')

In [5]:
"""
Based on the correlations:
'Resolving Insolvency' appears to be the most highly correlated, 
I've included other highly correlated variables below.
"""

NUM = 1000
res = []
for e in df3.columns:
     if df3[e].isna().sum() < NUM:
            res.append(e)
df4 = df3[res]
df4 = df4.corr().abs().sort_values(by='GDP/Cap(USD)', ascending=False)
df4.index[1:10]

Index(['Resolving insolvency (DB04-14 methodology) - Score',
       'Resolving insolvency: Recovery rate (cents on the dollar)',
       'Resolving insolvency: Recovery rate (cents on the dollar) - Score',
       'Trading across borders: Documents to export (number) (DB06-15 methodology) - Score',
       'Trading across borders: Documents to export (number) (DB06-15 methodology)',
       'Trading across borders: Documents to import (number) (DB06-15 methodology) - Score',
       'Resolving insolvency: Outcome (0 as piecemeal sale and 1 as going concern)',
       'Trading across borders: Documents to import (number) (DB06-15 methodology)',
       'Trading across borders (DB06-15 methodology) - Score'],
      dtype='object')

# 3. Chocolate Nobel question

In this repository is the academic paper `chocolate_nobel.pdf`. 

Explain in 3 paragraphs why this paper's conclusions are bad statistics.

In [5]:
"""

It seems difficult not to wonder why we don't see any representation from cacao producing countries. 
The only exception being Brazil, but for comparison: Brazil produced only 176,000 tons this year vs 2,154,000 tons out of the Ivory Coast.
    
So why don't we see Ivory Coast winning Nobels? Surely even when accounting for other factors that might limit their representation, we would still anticipate such countries to have won at least once. 

Especially If the studies proposal true that:
"The slope of the regression line allows us to estimate that it would take about 0.4 kg of chocolate per capita per year to increase the number of Nobel laureates in a given country by 1. "

If so, then a country with as much cocao as Côte d'Ivoire would be expected to see effect equal to or greater than those described in the proposal. 
(By logic of: Higher dose of the nutrients to which the increase in cognitive function is attributed)

But perhaps the flaw in this argument, and the arguments of the paper itself is the idea that: 
The distribution of nobel prizes is a sufficiently mechanical/objective process, as to have an unbiased distribution which can be meaingfully studied by statistical analysis.

"""

'\n\nIt seems difficult not to wonder why we don\'t see any representation from cacao producing countries. \nThe only exception being Brazil, but for comparison: Brazil produced only 176,000 tons this year vs 2,154,000 tons out of the Ivory Coast.\n    \nSo why don\'t we see Ivory Coast winning Nobels? Surely even when accounting for other factors that might limit their representation, we would still anticipate such countries to have won at least once. \n\nEspecially If the studies proposal true that:\n"The slope of the regression line allows us to estimate that it would take about 0.4 kg of chocolate per capita per year to increase the number of Nobel laureates in a given country by 1. "\n\nIf so, then a country with as much cocao as Côte d\'Ivoire would be expected to see effect equal to or greater than those described in the proposal. \n(By logic of: Higher dose of the nutrients to which the increase in cognitive function is attributed)\n\n'